# Claude CLI - Gestion des Sessions

**Module :** Vibe-Coding / Claude Code / Notebooks CLI  
**Niveau :** Debutant  
**Duree :** 25 min  
**Prerequis :** Notebook 01 complete

## Objectifs d'Apprentissage

A la fin de ce notebook, vous saurez :

- [ ] Comprendre le concept de sessions Claude et **pourquoi elles sont essentielles**
- [ ] Continuer une conversation avec `-c` pour des **interactions iteratives**
- [ ] Reprendre une session specifique par son ID
- [ ] Creer un fork de session pour **explorer des alternatives**
- [ ] Scripter des conversations multi-tours pour **l'automatisation**

> **Cas d'usage typiques** : Debug iteratif, exploration de solutions, prototypage conversationnel

---

## 1. Configuration

In [ ]:
import sys
import subprocess
sys.path.insert(0, 'helpers')

SIMULATION_MODE = False

from claude_cli import (
    run_claude, verify_installation, print_response,
    run_claude_continue, run_claude_command
)
import claude_cli
claude_cli.SIMULATION_MODE = SIMULATION_MODE

if verify_installation():
    print("Claude CLI pret")
else:
    print("Claude CLI non installe")
    if SIMULATION_MODE:
        print("(Mode simulation actif - les demos fonctionneront quand meme)")

## 2. Concept de Sessions

Par defaut, chaque appel `claude -p` est **sans etat** : Claude ne se souvient pas des echanges precedents.

Les **sessions** permettent de maintenir un contexte conversationnel :

```
┌─────────────────────────────────────────────────────┐
│                    Session                          │
│  ┌───────┐   ┌───────┐   ┌───────┐   ┌───────┐    │
│  │ Msg 1 │ → │ Msg 2 │ → │ Msg 3 │ → │ Msg 4 │    │
│  └───────┘   └───────┘   └───────┘   └───────┘    │
│       ↓           ↓           ↓           ↓        │
│  Claude se souvient de tout le contexte            │
└─────────────────────────────────────────────────────┘
```

### Comment ca fonctionne ?

1. **Stockage local** : Les sessions sont stockees dans `~/.claude/sessions/`
2. **ID unique** : Chaque session a un identifiant hexadecimal (ex: `a1b2c3d4`)
3. **Contexte cumule** : Chaque nouveau message inclut l'historique complet

> **Attention aux couts** : Une session longue de 20 messages renvoie TOUT l'historique a chaque appel. Cela peut consommer beaucoup de tokens.

## 3. Continuer une Conversation (-c)

Le flag `-c` (continue) reprend la derniere conversation :

```bash
claude -p "Premiere question"    # Cree une nouvelle session
claude -c "Question de suivi"    # Continue la meme session
```

> **Important** : Le flag `-c` reprend la **derniere** conversation, pas une conversation specifique.
> 
> Si vous executez `claude -p "autre question"` entre deux `-c`, vous perdez le contexte !
> 
> ```bash
> claude -p "Question A"      # Session 1
> claude -c "Suite de A"      # Session 1 (OK)
> claude -p "Question B"      # Session 2 (NOUVELLE)
> claude -c "Suite de A ?"    # Session 2 (pas Session 1 !)
> ```

In [ ]:
# Message 1 : Etablir un contexte
stdout, stderr, code = run_claude(
    "Je vais te poser des questions sur Python. D'accord ?"
)
print("=== Message 1 ===")
print_response(stdout, stderr, code)

In [ ]:
# Message 2 : Continuer avec -c
# Equivalent CLI : claude -c "Quelle est la difference entre list et tuple ?"
stdout, stderr, code = run_claude_continue(
    "Quelle est la difference entre list et tuple ?"
)

print("=== Message 2 (suite) ===")
print_response(stdout, stderr, code)

In [ ]:
# Message 3 : Encore une suite
# Equivalent CLI : claude -c "Et pour un dict vs un set ?"
stdout, stderr, code = run_claude_continue(
    "Et pour un dict vs un set ?"
)

print("=== Message 3 (suite) ===")
print_response(stdout, stderr, code)

## 4. Sessions avec ID

Chaque session a un identifiant unique. Vous pouvez le specifier pour reprendre une session specifique :

```bash
# Voir les sessions disponibles
claude /sessions

# Reprendre une session par ID (format hexadecimal)
claude --session-id abc123def456 -p "Suite de la conversation"

# Afficher l'historique d'une session
claude --session-id abc123def456 --print-history
```

### Format des IDs de session

Les IDs sont des chaines hexadecimales de 12-16 caracteres, par exemple :
- `a1b2c3d4e5f6`
- `session_789abc`

> **Tip** : Notez l'ID d'une session importante pour pouvoir la reprendre plus tard, meme apres avoir lance d'autres conversations.

In [ ]:
# Lister les sessions recentes
# Equivalent CLI : claude /sessions
stdout, stderr, code = run_claude_command("/sessions")

print("=== Sessions Disponibles ===")
if code == 0:
    print(stdout if stdout else "Aucune session disponible")
else:
    print(f"Commande non supportee ou erreur: {stderr}")

## 5. Fork de Session

Le **fork** permet de creer une branche a partir d'une session existante :

```
Session originale:  A → B → C
                         ↓
Fork:                    B → D → E  (nouvelle branche)
```

### Pourquoi utiliser un fork ?

1. **Explorer des alternatives** : Tester deux approches sans perdre la premiere
2. **Rollback** : Revenir a un etat precedent de la conversation
3. **Comparaison** : Demander la meme chose differemment pour comparer

### Syntaxe

```bash
# Continuer normalement
claude -c "Suite normale"

# Creer un fork (nouvelle branche)
claude -c --fork-session "Alternative a explorer"
```

> **Note** : Le fork cree une NOUVELLE session. L'originale reste intacte.

Utile pour explorer des alternatives sans perdre la conversation originale.

In [ ]:
# Creer une conversation de base
stdout, stderr, code = run_claude(
    "Je developpe une API REST en Python. Quel framework recommandes-tu ?"
)
print("=== Question initiale ===")
print_response(stdout, stderr, code)

In [ ]:
# Continuer normalement
# Equivalent CLI : claude -c "Quels sont les avantages de FastAPI ?"
stdout, stderr, code = run_claude_continue(
    "Quels sont les avantages de FastAPI ?"
)
print("=== Suite normale ===")
print_response(stdout, stderr, code)

In [ ]:
# Fork pour explorer Flask a la place
# Equivalent CLI : claude -c --fork-session "Et si je choisis Flask a la place ?"
stdout, stderr, code = run_claude_continue(
    "Et si je choisis Flask a la place ?",
    fork=True
)
print("=== Fork (alternative Flask) ===")
if code == 0:
    print_response(stdout, stderr, code)
else:
    print("Fork non supporte dans cette version ou erreur")
    if stderr:
        print(f"Details: {stderr}")

## 6. Script Multi-Tours

Voici comment creer une conversation programmatique avec plusieurs echanges :

In [ ]:
def conversation_multi_tours(messages: list) -> list:
    """
    Execute une conversation multi-tours.
    
    Args:
        messages: Liste de prompts a envoyer en sequence.
        
    Returns:
        Liste des reponses.
    """
    responses = []
    
    for i, msg in enumerate(messages):
        if i == 0:
            # Premier message : nouvelle session
            stdout, stderr, code = run_claude(msg)
        else:
            # Messages suivants : continuer
            stdout, stderr, code = run_claude_continue(msg)
        
        responses.append({
            "question": msg,
            "response": stdout,
            "success": code == 0
        })
    
    return responses

In [ ]:
# Exemple de conversation
conversation = [
    "Je veux creer un jeu en Python. Par quoi commencer ?",
    "Quel est le framework le plus simple pour debuter ?",
    "Peux-tu me donner un exemple de code minimal ?"
]

# Executer (attention : consomme des tokens)
# Decommentez pour executer
# results = conversation_multi_tours(conversation)
# for r in results:
#     print(f"Q: {r['question'][:50]}...")
#     print(f"R: {r['response'][:200]}...\n")

## 7. Exercices Pratiques

In [ ]:
# EXERCICE 1 : Creez une conversation de 3 messages sur un sujet technique
#
# Objectif : Pratiquer l'utilisation de -c pour maintenir le contexte
#
# Instructions :
# 1. Message 1 : Posez une question generale (ex: "Explique les decorateurs Python")
# 2. Message 2 : Demandez des details avec run_claude_continue()
# 3. Message 3 : Demandez une application avec run_claude_continue()
#
# Conseil : Verifiez que la reponse 3 fait reference aux reponses precedentes !

# === Message 1 ===
# stdout, stderr, code = run_claude("Votre question ici")
# print_response(stdout, stderr, code)

# === Message 2 (avec -c via helper) ===
# stdout, stderr, code = run_claude_continue("Votre question de suivi")
# print_response(stdout, stderr, code)

# === Message 3 (avec -c via helper) ===
# stdout, stderr, code = run_claude_continue("Votre question finale")
# print_response(stdout, stderr, code)

In [ ]:
# EXERCICE 2 : Utilisez la fonction conversation_multi_tours
# pour creer une conversation sur les design patterns
#
# Objectif : Automatiser une conversation educative

design_pattern_conv = [
    "Quels sont les 3 design patterns les plus utilises en Python ?",
    "Explique le pattern Singleton avec un exemple Python",
    "Quels sont les inconvenients du Singleton ?",
    "Quelle alternative recommandes-tu ?"
]

# === Execution ===
# Decommentez les lignes suivantes pour executer (consomme des tokens)
#
# results = conversation_multi_tours(design_pattern_conv)
#
# print("\n" + "="*50)
# print("RESUME DE LA CONVERSATION")
# print("="*50)
# for r in results:
#     print(f"\nQ: {r['question'][:50]}...")
#     print(f"R: {r['response'][:200]}...")
#     print("-"*30)

## 8. Bonnes Pratiques

### Quand utiliser les sessions ?

| Situation | Recommandation |
|-----------|----------------|
| Question isolee | `claude -p` simple |
| Exploration iterative | Sessions avec `-c` |
| Debug progressif | Sessions avec `-c` |
| Comparaison d'approches | Fork de session |
| Script automatise | Nouvelle session a chaque run |

### Conseils

1. **Limitez la longueur** : Les sessions tres longues consomment beaucoup de tokens
2. **Resumez** : Demandez a Claude de resumer la conversation si elle devient longue
3. **Nouvelle session** : En cas de changement de sujet majeur, creez une nouvelle session

### Estimation des couts

| Longueur session | Tokens/message (approx.) | Cout estime* |
|------------------|--------------------------|--------------|
| 5 messages | ~2,000 tokens | $0.01 |
| 15 messages | ~8,000 tokens | $0.04 |
| 30 messages | ~20,000 tokens | $0.10 |

*Couts approximatifs avec Claude Sonnet. Varient selon le modele et la longueur des reponses.

> **Tip** : Pour les sessions longues, demandez periodiquement :
> "Resume notre conversation en 3 points cles" pour reduire le contexte.

## 9. Resume

Dans ce notebook, nous avons appris :

- **Sessions** : Maintiennent le contexte entre les echanges
- **`-c`** : Continue la derniere conversation
- **`--session-id`** : Reprend une session specifique
- **`--fork-session`** : Cree une branche alternative
- **Scripts multi-tours** : Automatiser des conversations

### Aide-memoire des commandes

```bash
# Commandes essentielles
claude -p "question"              # Nouvelle session
claude -c "suite"                 # Continuer derniere session
claude --session-id ID -p "msg"   # Reprendre session specifique
claude -c --fork-session "alt"    # Creer un fork

# Gestion des sessions
claude /sessions                  # Lister les sessions
claude /status                    # Etat courant
```

### Points cles a retenir

1. Chaque `-p` sans `-c` cree une NOUVELLE session
2. Les sessions longues coutent cher en tokens
3. Utilisez les forks pour explorer sans risque

### Prochaine etape

Dans le notebook suivant, nous verrons comment **referencer des fichiers** avec les @-mentions.

-> [03-Claude-CLI-References.ipynb](03-Claude-CLI-References.ipynb)